# Utilizando a API funcional do keras para criar uma rede neural MLP

Nesse notebook é utilizado o dataset [Concrete Compressive Strength Data Set](https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength), onde as features são:

* Aglomerante: Cement (kg in a m3 mixture)
* Escória de Alto Forno: Blast Furnace Slag (kg in a m3 mixture)
* Cinzas volantes: Fly Ash (kg in a m3 mixture)
* Água: Water (kg in a m3 mixture)
* Superplastificante: Superplasticizer (kg in a m3 mixture)
* Agregado graúdo: Coarse Aggregate (kg in a m3 mixture)
* Agregado fino: Fine Aggregate (kg in a m3 mixture)
* Idade: Age (Day (1~365))

O objetivo é com essas features obter a resistência à compressão do concreto, em inglês: Concrete compressive strength (MPa).

Primeiro são importadas as bibliotecas.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import numpy as np

import pandas as pd

np.random.seed(0)
tf.random.set_seed(0)


O dataset é aberto usando o pandas. O formato original é em xls, então também é especificada a planilha aser aberta, a primeira. As features são renomeadas para facilitar as referências.

São definidos os dataframes com features e variável de saída.

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
df = pd.read_excel(url, sheet_name=0)

df.columns = ['cement', 'slag', 'ash', 'water', 'superplasticizer', 'coarse_aggregate', 'fine_aggregate', 'age', 'compressive_strength']

X = (df.drop(['compressive_strength'], axis=1))
y = (df['compressive_strength'])

O dataset é dividido em treino e teste.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [4]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()

X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)

São definidas duas variáveis para a média e o desvio padrão da variável alvo. 

In [5]:
mean = y_train.mean()
std = y_train.std()

Uma função para ciração da rede é definida utilizando a API funcional do Keras. Isso permite com que o resultado final da rede seja multiplicado pelo desvio padrão e somado à média. Dessa forma os pesos são ajustados à variàvel objetivo padronizada.

In [6]:
def create_model():
  x = Input(shape=(X_train.shape[1],))
  hidden1 = Dense(20, activation='relu')(x)
  hidden2 = Dense(100, activation='relu')(hidden1)
  hidden3 = Dense(20, activation='relu')(hidden2)
  hidden4 = Dense(1)(hidden3)
  output = hidden4 * std + mean
  model = Model(inputs=x, outputs=output)
  opt = keras.optimizers.Adam(learning_rate=1e-3)
  model.compile(loss="mean_absolute_error", optimizer=opt)  
  
  return model

O modelo é criado e as camadas são mostradas.

In [7]:
model = create_model()

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense (Dense)                (None, 20)                180       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                2020      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
_________________________________________________________________
tf.math.multiply (TFOpLambda (None, 1)                 0         
_________________________________________________________________
tf.__operators__.add (TFOpLa (None, 1)                 0     

Um wraper é criado para utilizar a função de validação cruzado do Scikit-learn.

In [9]:
from keras.wrappers.scikit_learn import KerasRegressor

estimator = KerasRegressor(build_fn=create_model, epochs=300, batch_size=300, verbose=0)

O score da validação cruzada é calculado.

In [10]:
print(-cross_val_score(estimator, X_train, y_train, cv=5, scoring='neg_mean_absolute_error').mean())

4.075838752319069


O score da validação cruzada no conjunto de treino é de 4.08 MPa.

Por fim, o modelo é treinado utilizando todo o conjunto de treino.

In [11]:
model.fit(x=X_train, y=y_train.values, epochs=300, batch_size=300)

Epoch 1/300
3/3 [==============================] - 1s 8ms/step - loss: 13.0678
Epoch 2/300
3/3 [==============================] - 0s 6ms/step - loss: 12.3589
Epoch 3/300
3/3 [==============================] - 0s 3ms/step - loss: 11.7695
Epoch 4/300
3/3 [==============================] - 0s 4ms/step - loss: 11.4504
Epoch 5/300
3/3 [==============================] - 0s 3ms/step - loss: 10.9317
Epoch 6/300
3/3 [==============================] - 0s 5ms/step - loss: 10.5465
Epoch 7/300
3/3 [==============================] - 0s 5ms/step - loss: 10.2404
Epoch 8/300
3/3 [==============================] - 0s 5ms/step - loss: 9.9319
Epoch 9/300
3/3 [==============================] - 0s 4ms/step - loss: 9.5963
Epoch 10/300
3/3 [==============================] - 0s 4ms/step - loss: 9.3915
Epoch 11/300
3/3 [==============================] - 0s 5ms/step - loss: 9.4176
Epoch 12/300
3/3 [==============================] - 0s 4ms/step - loss: 9.1122
Epoch 13/300
3/3 [==============================] - 0s

O erro médio absoluto final é então calculado.

In [12]:
mean_absolute_error(y_test, model(X_test))

3.8446156233084365

O modelo prevê a resistência à compressão do concreto com um erro médio absoluto de 3.845 MPa para o conunto de teste.